# Assignment 7: Transfer Learning


The goal of this exercise is to learn how to use pre-trained networks in transfer learning tasks.
We will make use of networks trained on ImageNet, and apply them to related problems, i.e., the classification of $10$ objects not contained in ImageNet.

## Dataset

For this exercise we use the  [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset that can be downloaded from the official website [here]({https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz}).
The dataset contains $60000$ color images of pixels size $32\times 32$ in $10$ classes: airplane, automobile, bird, cat, deer, dog, frog, horse, ship and truck, with $6000$ images per class.

### Task 1: Data Transformation

We need to instantiate a proper `torchvision.transform` instance to create the same input structure as used for training our network.
We need to combine 4 transforms, which can be compiled from the PyTorch website: https://pytorch.org/vision/stable/models.html

1. We need to resize the image such that the shorter side has size 256.
2. We need to take the center crop of size $224\times224$ from the image.
3. We need to convert the image into a tensor (including pixel values scaling)
4. We need to normalize the pixel values with mean $(0.485, 0.456, 0.406)$ and standard deviation $(0.229, 0.224, 0.225)$.

Since we will use networks pre-trained on ImageNet, we need to perform the exact same transform as used for ImageNet testing.

In [1]:
import torch
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

imagenet_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

### Task 2: Dataset Loading

We here use the [torchvision.datasets.CIFAR10](https://pytorch.org/vision/0.12/generated/torchvision.datasets.CIFAR10.html) dataset interface for processing images. 
You can use the `train` argument or flag to distinguish between training and test set.

This task consists of two parts:

1. Create two datasets, one for the training set, one for the test set. Use the transform defined above.
2. Once the datasets are created, create two data loaders, one for training set, one for test set. Use a proper value of the batch-size $B$.

In [2]:
trainset = torchvision.datasets.CIFAR10(
  root = "./CIFAR-10", train=True, transform=imagenet_transform, download=True
)

testset = torchvision.datasets.CIFAR10(
  root = "./CIFAR-10", train=False, transform=imagenet_transform, download=True
)
print(len(testset))

Files already downloaded and verified
Files already downloaded and verified
10000


In [3]:
B = 256
trainloader = torch.utils.data.DataLoader(trainset, shuffle=True, batch_size = B)
testloader = torch.utils.data.DataLoader(testset, shuffle=False, batch_size = B)

### Test 1: Data Size and Types

We check that all input images are `torch.tensors` of size $3\times224\times224$ and of type `torch.float` and that all labels are of type `int`.

Note: the sanity check is only performed on the test set.

In [4]:
# for x, t in testset:
#   assert isinstance(x, torch.Tensor)
#   assert isinstance(t, int)
#   assert x.shape==(3,224,224)
#   assert x.dtype==torch.float

## Deep Feature Extraction

We will use a pre-trained network available in `PyTorch`. 
Particularly, we will use a ResNet-50 architecture, but other architectures can also be tested. 
Fortunately, PyTorch provides simple interfaces to obtain pre-trained models, e.g., using the `torchvision.models.resnet50` interface function.

In order to use the networks in a different dataset, we need to change their outputs. 
There are several possibilities on how to achieve that, and you have the freedom to choose. 

For your reference, the implementation of the `forward` function of ResNet networks (including ResNet-50) can be found here: https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py#L266

You can also check if other networks perform better, for example, deeper ResNet topologies.
Be aware that the strategy to replace the last fully-connected layer might not work in other network topologies, only in residual networks.

### Task 3: Pre-trained Network Instantiation

Instantiate two pre-trained networks of type ResNet-50.

1. Freeze the feature layers of the first network.

Note: Make use the `old TorchVision Interface` to load your pre-trained network. Here is the link: https://pytorch.org/vision/0.12/models.html 

In [5]:
# instantiate the first pre-trained resnet 50 network
network_1 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
# print(network_1)
# Make sure to freeze all the layers of the network.
for param in network_1.parameters():
    param.requires_grad = False

# instantiate the second pre-trained resnet 50 network (optinally)
network_2 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

### Task 4: Network Implementation

We want to modify the network such that we extract the logits for the 10 classes from CIFAR-10 from the last fully-connected layer of the network.

Implement a function that:
1. Replaces the current last linear layer of the pre-trained network with a new linear layer that has $O$ units ($O$ represents the number of classes in our dataset).
2. Initialize the weights of the new linear layer using Xavier's method **(Optional)**.

Note: Use `torch.nn.init.xavier_uniform_` function to initialize the weights of the new linear layer.

In [6]:
def replace_last_layer(network, O=10):
  # replace the last linear layer with the new layer
  network.fc = torch.nn.Linear(network.fc.in_features, O)
  # initialize the weights of the new linear layer
  torch.nn.init.xavier_uniform_(network.fc.weight)
  return network

### Test 2: Last layer dimensions

This test ensures that the function return a network having the correct number of input and output units in the last layer.

In [7]:
O = 10
for network in (network_1, network_2):
    new_model = replace_last_layer(network, O=O)
    assert new_model.fc.out_features == O
    assert new_model.fc.in_features == 2048

## Network Training
Implement a function that takes all necessary parameters to run a training on a given dataset. 
Select the optimizer to be `torch.optim.SGD` and `torch.nn.CrossEntropyLoss` as the loss function. 
The test set will be used as the validation set.

### Task 5: Training and Evaluation Loop

Implement a training loop over a specific number of epochs (10) with a learning rate of $\eta=0.001$ and momentum of $\mu = 0.9$. 
Make sure that you train on the training data only, and `not` on the validation data.
In each loop, compute and print the training loss, training accuracy, validation loss and validation accuracy. 

In [8]:
def train_eval(model, epochs=10, eta=1e-3, miu=0.9):
    loss = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=eta, momentum=miu)

    device = torch.device("cuda")
    model = model.to(device)
    model.train()

    train_loss, train_acc = [], []
    test_loss, test_acc = [], []

    for epoch in range(epochs):
        loss_train, acc_train = 0, 0
        # training process
        for x,t in trainloader:
            z = model(x.to(device))
            optimizer.zero_grad()
            J = loss(z, t.to(device))
            loss_train += J.item()
            acc_train += torch.sum(torch.argmax(z, dim=1) == t.to(device)).item()
            J.backward()
            optimizer.step()

        train_loss.append(loss_train / len(trainset))
        train_acc.append(acc_train / len(trainset))
        
        # testing process
        with torch.no_grad():
            loss_test, acc_test = 0, 0
            for x,t in testloader:
                z = model(x.to(device))
                # compute validation loss
                J_test = loss(z, t.to(device))
                loss_test += J_test.item()
                # compute validation accuracy
                acc_test += torch.sum(torch.argmax(z, dim=1) == t).item()

            test_loss.append(loss_test / len(testset))
            test_acc.append(acc_test / len(testset))

        # print accuracies and losses for current epoch
        print(f"Epoch:{epoch}, \
                Train_Loss:{train_loss[-1]},Train_Acc:{train_acc[-1]} | \
                Test_Loss:{test_loss[-1]},Test_Acc:{test_acc[-1]}")

### Task 6: Network Fine-Tuning with Frozen Layers

Create a network that has feature layers frozen with $10$ output units. 
Fine-tune the created network on our CIFAR-10 data using the previous function.

In [9]:
torch.cuda.empty_cache()
network_with_frozen_layers = replace_last_layer(network_1, 10)
print(network_with_frozen_layers.fc)
train_eval(network_with_frozen_layers)

Linear(in_features=2048, out_features=10, bias=True)
20
40
60
80
100
120
140
160
180


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument target in method wrapper_CUDA_nll_loss_forward)

### Task 7 (Optional): Network Fine-Tuning without Frozen Layers 

Create a network from the second pre-trained network with $10$ output units. 
Fine-tune the created network on our CIFAR-10.

Note:

  * The fine-tuning of the network can take a long time when the layers are not frozen.

In [ ]:
network_normal = ...
train_eval(...)

## Plotting

Finally, we want to plot the confusion matrix of the test set.
For this, we need to compute the predictions for all of our test samples, and the list of target values.
Finally, we can make use of the `sklearn.metrics.confusion_matrix` to compute the confusion matrix.
You can utilize `sklearn.metrics.ConfusionMatrixDisplay` for displaying the confusion matrix, or `pyplot.imshow` and adding the according labels.

Note:

  * The documentation for the confusion matrix can be found here: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
  * The interface and an example for the `ConfusionMatrixDisplay` can be found here: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html

### Task 8: Confusion Matrix Plotting

Plot the confusion matrix for the fine-tuned network with frozen layers.
Optionally, also plot the confusion matrix for the second fine-tuned network.

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

network_with_frozen_layers.eval()
# compute predictions and collect targets
predictions = []
targets = []
with torch.no_grad():
    for images, targets in testloader:
        outputs = network_with_frozen_layers(images)
        _, predicted = torch.max(outputs.data, 1)
        predictions.append(predicted.cpu().numpy())
        targets.append(targets.cpu().numpy())

# compute confusion matrix
matrix = confusion_matrix(targets, predictions)

# plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=list(range(O)))
disp.plot()